## Literature analysis with non-negative matrix factorization topic model 

Project: Scientific Anaylsis <br>
Documentation: Master Thesis <br>
Author: Tanyel Tunçer <br>
Supervisor: <br> 
Selected Database: <br> <br>
Date of analysis: 
<br>


### Data Preparation 

Step 1: Define a search string <br>
Step 2: Query scientific data based on the defined search string  <br>
Step 3: Export results <br>
Step 4: Import data from local to model
<br>

### Analysis Instructions

Step 1: Import data <br>
Step 2: Pre-process imported data <br>
Step 3: Exploratory data analysis <br>
Step 4: Define model parameter <br>
Step 5: Calculate topic coherence Cv <br>
Step 7: Execute non-negative matrix factorization algoritmn
<br>

### Technical Preparation for model execution 

Install following version before importing libaries <br>
Note: All results from the thesis are identified based on the listed libary versions, other version can result in different result <br> <br>
pandas: 1.2.4  <br>
numpy: 1.19.2 <br>
nltk: 3.7 <br>
plotly: 5.8.0 

In [1]:
import pandas as pd 
import numpy as np
import re
import os

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

import plotly.express as px

from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

pd.set_option("display.max_columns", None)
pd.get_option("display.max_rows")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tanye\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tanye\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


60

## 1. Data

In [2]:
import os

# get current working directory
cwd = os.getcwd()

#get files in directory
files = os.listdir(cwd) 

print(files)

['.ipynb_checkpoints', 'literature_analysis_tanyel_tuncer1.ipynb', 'research - Copy.xlsx', 'research.xlsx', 'scopus.csv', 'wordclouds.xlsx', '~$research.xlsx']


In [3]:
#Change data path. Data will be automatically imported from the given data path. 
text = pd.read_csv("C:/Users/tanye/Desktop/Literature Review/Literature 1/scopus.csv", sep = ",")

#text = pd.read_csv(document_dir, delimiter = ",")
text.head()

,Authors,Author full names,Author(s) ID,Titles,Year,Source title,DOI,Cited by,Link,Abstract,Author Keywords,Document Type,Source
0,Karthik Pai B.H.; Pai V.; Devidas; Deeksh S.N....,"Karthik Pai, B.H. (57214719472); Pai, Vasudeva...",57214719472; 57202778194; 57210970005; 5721160...,A prologue of git and SVN,2019,International Journal of Engineering and Advan...,10.35940/ijeat.A9451.109119,0,https://www.scopus.com/inward/record.uri?eid=2...,Version Control Software or Revision Control S...,Control tools; Git; SVN; Version Control,Article,Scopus
1,Subhani S.,"Subhani, Sohail (57469826700)",57469826700,Is Open Source Software the Future of Software...,2020,International Journal of Business,10.30845/ijbht.v10n1p2,0,https://www.scopus.com/inward/record.uri?eid=2...,"Just a few years ago, it would have been diffi...",Open Source Software; Proprietary Software; So...,Article,Scopus
2,Comino S.; Manenti F.M.; Parisi M.L.,"Comino, Stefano (8578610100); Manenti, Fabio M...",8578610100; 35997145900; 22981096300,From planning to mature: On the success of ope...,2007,Research Policy,10.1016/j.respol.2007.08.003,75,https://www.scopus.com/inward/record.uri?eid=2...,Open source is an example of user-centric inno...,Development status; Intended audience; License...,Article,Scopus
3,Amrollahi A.; Khansari M.; Manian A.,"Amrollahi, Alireza (56258799900); Khansari, Mo...",56258799900; 36613129200; 35113563700,Success of Open Source in Developing Countries...,2014,International Journal of Open Source Software ...,10.4018/ijossp.2014010103,3,https://www.scopus.com/inward/record.uri?eid=2...,Open Source approach has been recognized as on...,Developing Countries; Mixed Method; Open Sourc...,Article,Scopus
4,Wasserman A.I.,"Wasserman, Anthony I. (7102134728)",7102134728,Community and commercial strategies in open so...,2013,IT - Information Technology,10.1515/itit.2013.1003,1,https://www.scopus.com/inward/record.uri?eid=2...,Summary: This paper describes the recent evolu...,Acm css; Collaboration in software development...,Article,Scopus


In [4]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Authors            674 non-null    object
 1   Author full names  674 non-null    object
 2   Author(s) ID       674 non-null    object
 3   Titles             674 non-null    object
 4   Year               674 non-null    int64 
 5   Source title       674 non-null    object
 6   DOI                629 non-null    object
 7   Cited by           674 non-null    int64 
 8   Link               674 non-null    object
 9   Abstract           674 non-null    object
 10  Author Keywords    446 non-null    object
 11  Document Type      674 non-null    object
 12  Source             674 non-null    object
dtypes: int64(2), object(11)
memory usage: 68.6+ KB


In [5]:
text.isnull().sum()

Authors                0
Author full names      0
Author(s) ID           0
Titles                 0
Year                   0
Source title           0
DOI                   45
Cited by               0
Link                   0
Abstract               0
Author Keywords      228
Document Type          0
Source                 0
dtype: int64

## 2. Data pre-processing

In [6]:
text["Cited by"] = text["Cited by"].fillna(0)

text.drop(text[text["Abstract"]=="[No abstract available]"].index, inplace=True)
#text = text[text["Cited by"] >= 150]

text.head()

,Authors,Author full names,Author(s) ID,Titles,Year,Source title,DOI,Cited by,Link,Abstract,Author Keywords,Document Type,Source
0,Karthik Pai B.H.; Pai V.; Devidas; Deeksh S.N....,"Karthik Pai, B.H. (57214719472); Pai, Vasudeva...",57214719472; 57202778194; 57210970005; 5721160...,A prologue of git and SVN,2019,International Journal of Engineering and Advan...,10.35940/ijeat.A9451.109119,0,https://www.scopus.com/inward/record.uri?eid=2...,Version Control Software or Revision Control S...,Control tools; Git; SVN; Version Control,Article,Scopus
1,Subhani S.,"Subhani, Sohail (57469826700)",57469826700,Is Open Source Software the Future of Software...,2020,International Journal of Business,10.30845/ijbht.v10n1p2,0,https://www.scopus.com/inward/record.uri?eid=2...,"Just a few years ago, it would have been diffi...",Open Source Software; Proprietary Software; So...,Article,Scopus
2,Comino S.; Manenti F.M.; Parisi M.L.,"Comino, Stefano (8578610100); Manenti, Fabio M...",8578610100; 35997145900; 22981096300,From planning to mature: On the success of ope...,2007,Research Policy,10.1016/j.respol.2007.08.003,75,https://www.scopus.com/inward/record.uri?eid=2...,Open source is an example of user-centric inno...,Development status; Intended audience; License...,Article,Scopus
3,Amrollahi A.; Khansari M.; Manian A.,"Amrollahi, Alireza (56258799900); Khansari, Mo...",56258799900; 36613129200; 35113563700,Success of Open Source in Developing Countries...,2014,International Journal of Open Source Software ...,10.4018/ijossp.2014010103,3,https://www.scopus.com/inward/record.uri?eid=2...,Open Source approach has been recognized as on...,Developing Countries; Mixed Method; Open Sourc...,Article,Scopus
4,Wasserman A.I.,"Wasserman, Anthony I. (7102134728)",7102134728,Community and commercial strategies in open so...,2013,IT - Information Technology,10.1515/itit.2013.1003,1,https://www.scopus.com/inward/record.uri?eid=2...,Summary: This paper describes the recent evolu...,Acm css; Collaboration in software development...,Article,Scopus


In [7]:
corpus = text.copy()

def remove_stopwords(tokens):
    text = [x for x in tokens if x not in nltk.corpus.stopwords.words("english")]
    return text

def remove_punctuation(tokens):
    import string
    text = " ".join([x for x in tokens if x not in string.punctuation])
    text = " ".join([x for x in tokens if x not in string.digits])
    return text

def remove_words(tokens):
    words = ["δ18o", "paper"]
    text = [x for x in tokens if x not in words]
    text = [x for x in text if len(x) >= 3]
    #text = [x for x in text if x != ","]
    return text

    
def get_pos_tag(pos_tag):
    if pos_tag in ["JJ", "JJR", "JJS"]:
        return wordnet.ADJ
    elif pos_tag in ["RB", "RBR", "RBS"]:
        return wordnet.ADV
    elif pos_tag in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
        return wordnet.VERB
    else: 
        return wordnet.NOUN

def lemmatize_words(tokens):
    pos_tags = nltk.pos_tag(tokens)
    words = " ".join([WordNetLemmatizer().lemmatize(i) for i in tokens])
    return words

def extract_noun (tokens):
    pos_tags = nltk.pos_tag(tokens)
    tags = ["NN", "NNS", "NNP", "NNPS"]
    words = [i[0] for i in pos_tags if i[1] in tags]
    return words
                      

In [8]:
corpus["Abstract"] = corpus["Abstract"].map(lambda x: x.lower())
corpus["Abstract"] = corpus["Abstract"].map(lambda x: RegexpTokenizer(r"\w+").tokenize(x))
corpus["Abstract"] = corpus["Abstract"].apply(lambda x: remove_words(x))
corpus["Abstract"] = corpus["Abstract"].map(lambda x: remove_stopwords(x))
corpus["Abstract"] = corpus["Abstract"].apply(lambda x: extract_noun(x))
corpus["Abstract"] = corpus["Abstract"].apply(lambda x: lemmatize_words(x))
print("Data pre-processed.")

Data pre-processed.


In [9]:
print("Original data: " + text["Abstract"][0])
print("")
print("Pre-processed data: " + corpus["Abstract"][0])

Original data: Version Control Software or Revision Control Software are the most important things in the world of software development. In this paper, we have described two version control tools: Git and Apache Subversion. Git comes as free and open source code management and version control system which is disseminated with the GNU general public license. Apache Subversion abbreviated as SVN is one amongst a software versioning and revision control systems given as open source under Apache License. Git design, its functionality, and usage of Git and SVN are discussed in this paper. The goal of this research paper is to accentuate on GIT and SVN tools, evaluate and compare five version control tools to ascertain their usage and efficacy. © BEIESP.

Pre-processed data: version control software revision control software thing world software development version control apache subversion git source code management version control system license apache subversion amongst software revision 

## 3. Exploratory Data Analysis

In [10]:
colors1 = ["#023373"]
colors2 = ["#023373", "#979797"]

In [11]:
def line(df, x, y, z):
    """
    Plot line chart.
    
    df = dataframe 
    x = categorical variable
    y = numerical variable 
    z = color_code
    
    """
    fig = px.line(
        data_frame = df,
        x = x, 
        y = y,
        color = z,
        color_discrete_sequence= colors2,
        template="simple_white",
        width=1000,        
        height=400,
        log_x = False
    )
        
    return fig.show()

In [12]:
def bar_chart(df, x, y):
    """
    Plot bar chart.
    
    df = dataframe
    x = categorical variable
    y = numerical variable 
    
    """
    fig = px.bar(
        data_frame = df,
        x = x,
        y = y,
        color_discrete_sequence=["#03658C"],
        template="simple_white",
        orientation = "v",
        text = y,
        facet_col = "Period",
        facet_col_wrap=2,
        facet_row_spacing = 0.15,        
        width=1000,        
        height=600
    )
    
    fig.update_layout(font=dict(family="Times New Roman",size=12),
                      xaxis={"categoryorder":"category ascending"}
                     )
    
    fig.for_each_annotation(lambda x: x.update(text = x.text.replace("Period=", "")))
    fig.update_yaxes(title_text = "", visible = False)
    fig.update_xaxes(title_text = "Topic")
    
    
    return fig.show()

In [13]:
def barh_chart(df, x, y, title):
    """
    Plot bar chart.
    
    df = dataframe
    x = categorical variable
    y = numerical variable 
    
    """
    fig = px.bar(
        data_frame = df,
        x = x,
        y = y,
        color_discrete_sequence=["#03658C"],
        template="simple_white",
        orientation = "h",
        text = x,
        width=1000,        
        height=800
    )
    
    fig.update_layout(font=dict(family="Times New Roman",size=12),
                      yaxis={'categoryorder':'max ascending'},
                      title={
                          'text': title,
                          'y':0.98,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'}
                     )
    
    fig.update_yaxes(title_text = "")
    fig.update_xaxes(title_text = "", visible = False)
    
    
    return fig.show()

In [14]:
def histogram(df, x, n_bins, title): 
    """
    Plot histogram.
    
    df = dataframe
    x = categorical variable
    n_bins = bin size 
    
    """
    fig = px.histogram(
        data_frame=df,
        x = x,
        marginal= "box",
        color_discrete_sequence=["#03658C"],
        nbins=n_bins,
        template="simple_white",
        width=1000,        
        height=500
    )
    
    fig.update_layout(
        font_family="Times New Roman", 
        title={
            'text': title,
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
    )

    
    return fig.show()

In [15]:
def scatter_plot(df):
    """
    Plot scatter plot.
    
    df = dataframe with x and y coordinates 
    
    """
    fig = px.scatter(
        data_frame = df,
        #color = topics,
        opacity = 0.5,
        title = None,
        color_discrete_sequence=["#023373", "#03588C", "#03658C", "#6CBAD9", "#F2F2F2" ],
        template="simple_white",
        orientation = "h",
        width=1000,        
        height=600
    )
    
    fig.update_layout(
        font_family="Times New Roman" 
    )
    
    return fig.show() 

In [16]:
def area(df, x, y, z):  
    """
    Plot area chart.
    
    df = dataframe 
    x = categorical variable
    y = numerical variable 
    z = topics
    
    """
    fig = px.area(
        data_frame = df,
        x = x,
        y = y,
        template = "simple_white",
        color_discrete_sequence = colors1,
        facet_col = z,
        facet_col_wrap=2,
        facet_row_spacing = 0.1,
        height = 1000,
        width = 1000
    )
    
    fig.update_layout(
        font_family="Times New Roman",
        showlegend=False
    )
    
    fig.for_each_annotation(lambda x: x.update(text = x.text.replace("variable=", "")))
    
    fig.update_xaxes(title_text = "")
    fig.update_yaxes(title_text = "")
    
    return fig.show()

In [17]:
def plot_top_words(model, feature_names, n_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        fig = px.bar(
            x = weights,
            y = top_features,
            text =np.round(weights, 2),
            orientation = "h",
            color_discrete_sequence=["#03658C"],
            template = "simple_white", 
            #title="Long-Form Input",
            title = "Topic " + str(topic_idx),
            height = 400,
            width = 400

        )



        fig.update_layout(
            font_family="Times New Roman",
            showlegend=False,
            yaxis={'categoryorder':'max ascending'},
            title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'}
        )
        
        fig.for_each_annotation(lambda x: x.update(text = x.text.replace("topic=", " ")))
        fig.update_yaxes(title_text = "")
        fig.update_xaxes(title_text = "", visible = False)
        
        fig.show()


In [18]:
def fig_2d(df, x, y, document, color, topic_order):
    fig = px.scatter(
        df, 
        x=x, 
        y=y,
        color = color, labels={"topic": "Themen"},
        hover_name = document,
        opacity = 0.85,
        template="simple_white",
        width=800,        
        height=500,
        color_discrete_sequence=["red", "green", "blue", "orange", "goldenrod", "magenta"],
        category_orders= topic_order
    )
    
    fig.update_layout(showlegend=True)


    fig.update_xaxes(visible = False)
    fig.update_yaxes(visible = False)

    return fig.show()       

In [19]:
def fig_3d(df, x, y, z, color):
    fig = px.scatter_3d(
        df, 
        x=x, 
        y=y,
        z=z,
        color = color,
        #hover_name = "Topic",
        opacity = 0.85,
        template="simple_white",
        width=1000,        
        height=800,
        color_continuous_scale = colors2
    )
    
    fig.update_layout(showlegend=False)


    fig.update_xaxes(visible = False)
    fig.update_yaxes(visible = False)

    return fig.show()    

In [58]:
def tm_dist_period(df):
    df = df.copy()
    df = df.groupby(["Year", "Topic"]).size()
    df = pd.DataFrame(df).reset_index().rename(columns={0:"Count"})

    df["Year"] = df["Year"].astype(int).astype(str)
    df["Topic"] = df["Topic"].astype(str)

    conditions = [
        (df["Year"] <= "1997"),
        (df["Year"] >= "1998") & (df["Year"] <= "2005"),
        (df["Year"] >= "2006") & (df["Year"] <= "2013"),
        (df["Year"] >= "2014")
    ]

    titles = ["a) betweeen 1977 and 1997",
              "b) betweeen 1998 and 2005", 
              "c) betweeen 2006 and 2013",
              "d) betweeen 2014 and 2020"]

    df["Period"] = np.select(conditions, titles)
    df = df.groupby(by = ["Period", "Topic"]).agg(Count = ("Count", np.sum)).reset_index()
    
    fig = bar_chart(df, df["Topic"].astype("int64"), df["Count"])
    
    return fig

In [59]:
def topic_distribution(df):
    topic_count = pd.DataFrame()

    topic_count["Topic"] = df.Topic
    topic_count = topic_count.groupby("Topic").agg(
        Total_Documents = ("Topic", np.size),
        Proportion = ("Topic", np.size))

    topic_count["Proportion"] = topic_count["Proportion"].apply(lambda x: round((x * 100) / len(corpus), 2))

    return topic_count.reset_index()

In [60]:
histogram(text, "Cited by", 150, "Histogramm der Anzahl der Zitationen")

In [23]:
top_journals = (text.groupby(by = ["Source title"]).size().reset_index(name="Totel Documents"))
top_journals = top_journals.sort_values("Totel Documents", ascending=False)

top_journals = top_journals[:15]
barh_chart(top_journals, y = "Source title", x = "Totel Documents", title = "Top 15 Journals")

In [49]:
histogram(text, "Year", (len(text["Year"].unique())), "Histogram of the publication years")

In [48]:
corpus["word_count"] = corpus["Abstract"].apply(lambda x: len(str(x).split()))
histogram(corpus, "word_count", 30, "Histogram of the number of words in an abstract")

In [50]:
top_words = pd.Series(' '.join(corpus["Abstract"]).split()).value_counts()[:30]
top_words = pd.DataFrame(top_words, columns = ["count"])

barh_chart(top_words, "count", top_words.index, "Top 30 words in the corpus")

In [51]:
def get_top_n_gram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    
    sum_words = bag_of_words.sum(axis=0).round(2)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

top_words = get_top_n_gram(corpus["Abstract"], 30)
df3 = pd.DataFrame(top_words, columns = ["n_gram", "count"])

barh_chart(df3, "count", "n_gram", "Top 30 words in the corpus")

In [28]:
def topic_popularity(model, df):
    topic_distribution = df.iloc[:, 5:]
    topic_distribution["Year"] = df.Year
    
    ytime = np.unique(df['Year'])
    
    topic_distributions_by_year = np.zeros([len(ytime), model.n_components])

    topic_distribution_by_year = topic_distribution.groupby(by = "Year").sum()
    topic_distribution_by_year = topic_distribution_by_year / np.sum(topic_distribution_by_year)
    topic_distribution_by_year = topic_distribution_by_year.reset_index().melt(id_vars = "Year")
    
    return topic_distribution_by_year

## 4. Model parameter

In [29]:
n_features = None
n_top_words = 10

k_min = 3
k_max = 30 + 1
k_step = 1

max_df = 0.95
min_df = 2


## 5. Calculate Topic Coherence Cv

In [30]:
def gensim_data(corpus):
    from gensim.models.coherencemodel import CoherenceModel
    from gensim.corpora.dictionary import Dictionary
   
    gensim_data = [i.split( ) for i in corpus]
    gensim_dict = Dictionary(gensim_data)

    gensim_dict.filter_extremes(no_below=3, no_above=0.95, keep_n=5000)

    gensim_corpus = [gensim_dict.doc2bow(text) for text in gensim_data]
   
    return gensim_data, gensim_dict, gensim_corpus
gensim_data, gensim_dict, gensim_corpus = gensim_data(corpus["Abstract"])


In [31]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.nmf import Nmf
from operator import itemgetter

coherence = []
n_topic = list(np.arange(k_min, k_max, k_step))
for i in n_topic:
    gensim_nmfModel = Nmf(
        corpus=gensim_corpus,
        num_topics=i,
        id2word=gensim_dict,
        chunksize=2000,
        passes=5,
        kappa=.1,
        minimum_probability=0.01,
        w_max_iter=300,
        w_stop_condition=0.0001,
        h_max_iter=100,
        h_stop_condition=0.001,
        eval_every=10,
        normalize=True,
        random_state=1
    )
  
    cm = CoherenceModel(
        model=gensim_nmfModel,
        texts=gensim_data,
        dictionary=gensim_dict,
        coherence= "c_v"
    )
  
    coherence.append((cm.get_coherence(), i, "NMF"))

In [57]:
nmf_coherence_values = pd.DataFrame(coherence, columns = ["Coherence value Cv", "Number of subjects", "Algorithm"])
plot_coherence = line(nmf_coherence_values, "Number of subjects", "Coherence value Cv", None)


## 7. Non-negative Matrix Factorization Topic Modell 

In [33]:
tfidf_vectorizer = TfidfVectorizer(max_df = max_df, 
                                   min_df = min_df,
                                   max_features = n_features, 
                                   ngram_range = (1,2)
                                  )


In [34]:
tfidf_dtm = tfidf_vectorizer.fit_transform(corpus["Abstract"])

In [35]:
nmf_tm = NMF(n_components = 5, 
             random_state=1,
             alpha=.1,
             solver = "mu",
             max_iter = 1000,
             beta_loss="frobenius",
             l1_ratio=.5
            )


In [36]:
nmf_tm = nmf_tm.fit(tfidf_dtm)

C:\Users\tanye\miniconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning:

The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).



In [37]:
V = tfidf_vectorizer.transform(corpus["Abstract"])
W = nmf_tm.components_
H = nmf_tm.fit_transform(V) 


print("Non-negative Matrix Factorization Modell")
print('V = {} x {}'.format(V.shape[0], V.shape[1]))
print('W = {} x {}'.format(W.shape[0], W.shape[1]))
print('H = {} x {}'.format(H.shape[0], H.shape[1]))


Non-negative Matrix Factorization Modell
V = 674 x 5501
W = 5 x 5501
H = 674 x 5


In [38]:
nmf_df = pd.DataFrame(
    {"Topic" : np.argmax(H, axis = 1), 
     "Terms" : corpus["Abstract"], 
     "Keywords" : corpus["Author Keywords"], 
     "Title" : corpus["Titles"], 
     "DOI" : corpus["DOI"], 
     "Year" : corpus["Year"],
     "Journal" : corpus["Source title"]})


In [39]:
nmf_df = nmf_df.reset_index()

topic_names = ["Topic " + str(i) for i in range(nmf_tm.n_components)]

df_document_topic = pd.DataFrame(np.round(H, 2), columns = topic_names)

nmf_df = nmf_df.join(df_document_topic)
nmf_df


,index,Topic,Terms,Keywords,Title,DOI,Year,Journal,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,0,1,version control software revision control soft...,Control tools; Git; SVN; Version Control,A prologue of git and SVN,10.35940/ijeat.A9451.109119,2019,International Journal of Engineering and Advan...,0.03,0.04,0.00,0.00,0.00
1,1,1,year envision source software application sour...,Open Source Software; Proprietary Software; So...,Is Open Source Software the Future of Software...,10.30845/ijbht.v10n1p2,2020,International Journal of Business,0.01,0.15,0.00,0.00,0.00
2,2,4,source example user innovation group user need...,Development status; Intended audience; License...,From planning to mature: On the success of ope...,10.1016/j.respol.2007.08.003,2007,Research Policy,0.04,0.04,0.00,0.00,0.12
3,3,4,source approach method software development co...,Developing Countries; Mixed Method; Open Sourc...,Success of Open Source in Developing Countries...,10.4018/ijossp.2014010103,2014,International Journal of Open Source Software ...,0.04,0.06,0.00,0.01,0.08
4,4,1,describes evolution business strategy company ...,Acm css; Collaboration in software development...,Community and commercial strategies in open so...,10.1515/itit.2013.1003,2013,IT - Information Technology,0.00,0.18,0.07,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,669,0,spam mail mail user service spam mail harm org...,Electronic mail (e-mail); internet; machine le...,The Development of the Open Machine-Learning-B...,10.1109/ACCESS.2021.3118901,2021,IEEE Access,0.03,0.02,0.00,0.00,0.00
670,670,0,application sippi matlab toolbox probability d...,A posteriori; A priori; Inversion; Non-linear;...,SIPPI: A Matlab toolbox for sampling the solut...,10.1016/j.cageo.2012.10.001,2013,Computers and Geosciences,0.07,0.00,0.04,0.00,0.00
671,671,0,factor process ligand hydration water site dis...,NaN,Gr Predictor: A Deep Learning Model for Predic...,10.1021/acs.jcim.2c00987,2022,Journal of Chemical Information and Modeling,0.07,0.00,0.02,0.00,0.00
672,672,3,number website data download text chemspider u...,JCAMP-DX; Open data; Open science; REST API; S...,The Open Spectral Database: An open platform f...,10.1186/s13321-016-0170-2,2016,Journal of Cheminformatics,0.07,0.00,0.00,0.12,0.00


In [40]:
topic_distribution(nmf_df)

,Topic,Total_Documents,Proportion
0,0,433,64.24
1,1,119,17.66
2,2,29,4.30
3,3,55,8.16
4,4,38,5.64


In [44]:
feature_names = tfidf_vectorizer.get_feature_names()

plot_top_words(nmf_tm, feature_names, 10)


In [45]:
area(topic_popularity(nmf_tm, nmf_df), "Year", "value", "variable")


In [61]:
tm_dist_period(nmf_df)


In [47]:
nmf_df.to_excel("C:/Users/tanye/Desktop/Literature Review/Literature 1/research-dataset.xlsx")

In [62]:
top_words

[('source software', 295),
 ('source code', 180),
 ('source license', 147),
 ('github com', 109),
 ('business model', 90),
 ('http github', 89),
 ('http github com', 87),
 ('license http', 73),
 ('software package', 58),
 ('software development', 54),
 ('software source', 51),
 ('software license', 51),
 ('mit license', 49),
 ('university press', 49),
 ('software tool', 43),
 ('data analysis', 42),
 ('http www', 39),
 ('licensee ltd', 39),
 ('oxford university', 39),
 ('oxford university press', 39),
 ('gpl license', 39),
 ('author oxford', 37),
 ('author oxford university', 37),
 ('data set', 36),
 ('gene expression', 36),
 ('state art', 35),
 ('management system', 33),
 ('case study', 32),
 ('availability implementation', 32),
 ('source project', 31)]